In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

# ✅ 국가별 코드 및 URL 타입 지정
target_list = [
    ("FX_USDVND", "베트남", "world"),
    ("FX_USDIDR", "인도네시아", "world"),
    ("FX_USDPHP", "필리핀", "world"),
    ("FX_USDCNY", "중국", "world"),
    ("FX_USDKRW", "한국", "domestic"),  # 한국은 URL이 다름
]

# ✅ 결과 리스트
all_data = []
headers = {"User-Agent": "Mozilla/5.0"}

for code, country, source in target_list:
    if source == "world":
        url = f"https://finance.naver.com/marketindex/worldDailyQuote.naver?fdtc=4&marketindexCd={code}&page=1&pageSize=500"
    else:
        url = f"https://finance.naver.com/marketindex/exchangeDailyQuote.naver?marketindexCd={code}&page=1&pageSize=500"

    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    table = soup.select_one("table.tbl_exchange")
    rows = table.select("tbody > tr")

    for row in rows:
        cols = row.select("td")
        if len(cols) < 2:
            continue
        raw_date = cols[0].text.strip()
        raw_close = cols[1].text.strip().replace(",", "")
        try:
            date_fmt = datetime.strptime(raw_date, "%Y.%m.%d").strftime("%Y-%m-%d")
            close = float(raw_close)
            all_data.append([country, code, date_fmt, close])
        except ValueError:
            continue

# ✅ DataFrame 생성
df_all = pd.DataFrame(all_data, columns=["국가", "환율코드", "날짜", "종가"])
print(df_all.head())


    국가       환율코드          날짜       종가
0  베트남  FX_USDVND  2025-04-29  26005.0
1  베트남  FX_USDVND  2025-04-28  26000.0
2  베트남  FX_USDVND  2025-04-26  26021.5
3  베트남  FX_USDVND  2025-04-25  26021.5
4  베트남  FX_USDVND  2025-04-24  26029.0


In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

# ✅ 타겟 URL
url = "https://finance.naver.com/marketindex/exchangeDailyQuote.naver?marketindexCd=FX_USDKRW&page=1&pageSize=500"
headers = {"User-Agent": "Mozilla/5.0"}

# ✅ 요청 및 파싱
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")
table = soup.select_one("table.tbl_exchange")
rows = table.select("tbody > tr")

# ✅ 데이터 수집
data = []
for row in rows:
    cols = row.select("td")
    if len(cols) < 2:
        continue
    raw_date = cols[0].text.strip()
    raw_close = cols[1].text.strip().replace(",", "")
    try:
        date_fmt = datetime.strptime(raw_date, "%Y.%m.%d").strftime("%Y-%m-%d")
        close = float(raw_close)
        data.append(["한국", "FX_USDKRW", date_fmt, close])
    except ValueError:
        continue

# ✅ DataFrame 생성
df_krw = pd.DataFrame(data, columns=["국가", "환율코드", "날짜", "종가"])

# ✅ 출력
print(df_krw.head())


   국가       환율코드          날짜      종가
0  한국  FX_USDKRW  2025-04-30  1430.8
1  한국  FX_USDKRW  2025-04-29  1434.0
2  한국  FX_USDKRW  2025-04-28  1437.0
3  한국  FX_USDKRW  2025-04-25  1439.5
4  한국  FX_USDKRW  2025-04-24  1432.0
